In [1]:
%matplotlib nbagg
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle

import nltk
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

chatbot_path = "/home/bi0max/projects/tutorials/chatbot"
if chatbot_path not in sys.path:
    sys.path.append(chatbot_path)

from chatbot.config import *
from chatbot.embed_data import *
from chatbot import embed_data
from chatbot.train import *

from keras.layers import Embedding, LSTM, Input, Dense
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
# read files with samples
path = os.path.join(DATA_DIR, "tokenized_from.pickle")
tokenized_from = pickle.load(open(path, "rb"))
path = os.path.join(DATA_DIR, "tokenized_to.pickle")
tokenized_to = pickle.load(open(path, "rb"))

# read matrix, index
embedding_matrix = pickle.load(open(PATHS["embedding_matrix"], "rb"))
word2index = pickle.load(open(PATHS["word2index"], "rb"))

In [20]:

print("Opening files...")
path = os.path.join(DATA_DIR, "tokenized_from_clean.pickle")
tokenized_from = pickle.load(open(path, "rb"))
path = os.path.join(DATA_DIR, "tokenized_to_clean.pickle")
tokenized_to = pickle.load(open(path, "rb"))

Opening files...


In [9]:
edi, do = create_model_input_output(tokenized_from[:10], tokenized_to[:10], word2index, PREPROCESSING_PARAMS)

In [15]:
do_list = []
for t in range(do.shape[1]):
    do_list.append(do[:, t, :])

In [18]:
do_list[0].shape

(10, 12004)

In [2]:
model = load_model(PATHS['model'])

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, 50)           0                                            
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 100)      1200400     encoder_inputs[0][0]             
                                                                 decoder_inputs[0][0]             
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 128), (None, 117248      embedding_1[0][0]                
__________

In [13]:
model = create_model_attention(PREPROCESSING_PARAMS, HPARAMS)

In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, 50)           0                                            
__________________________________________________________________________________________________
shared_embedding (Embedding)    (None, 50, 100)      1200400     encoder_inputs[0][0]             
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 256)          0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 50, 256)      234496      shared_embedding[0][0]           
__________________________________________________________________________________________________
repeat_vec